In [ ]:
!nvidia-smi

Wed Jul  3 15:16:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              25W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!sudo apt install build-essential python3-dev libopenblas-dev -y

In [ ]:
!pip install torch ninja

In [ ]:
!pip install -U git+https://github.com/richlukich/MinkowskiEngine -v --no-deps \
                          --config-settings="--install-option=--force_cuda" \
                          --config-settings="--install-option=--blas=openblas"

In [ ]:
!pip install faiss-gpu
!pip install loguru
!pip install fuzzywuzzy
!pip install paddlepaddle-gpu
!pip install paddleocr
!pip install numpy-quaternion
!pip install dvc
!pip install dvc_gdrive

In [ ]:
import torch
import MinkowskiEngine as ME

In [ ]:
DATA_ROOT = '/kaggle/input/itlp-campus-dataset/public'

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('/kaggle/input/itlp-campus-dataset/public/train.csv').drop(columns='Unnamed: 0')
train_df

,timestamp,track,front_cam_ts,back_cam_ts,tx,ty,tz,qx,qy,qz,qw,front_cam_text,back_cam_text
0,1676034260851770,2023-02-10-08-04-19-twilight,1676034260821540,1676034260865870,-23.839826,-17.655232,-1.474530,0.013051,0.028187,0.003870,0.999510,[],[]
1,1676034268718093,2023-02-10-08-04-19-twilight,1676034268687989,1676034268708533,-19.060815,-19.003152,-2.513494,0.017344,0.019243,-0.121944,0.992199,[],[]
2,1676034274365801,2023-02-10-08-04-19-twilight,1676034274404483,1676034274382407,-14.289875,-20.252643,-2.563523,0.019104,0.025107,0.103990,0.994078,[],['KE']
3,1676034280013503,2023-02-10-08-04-19-twilight,1676034280004376,1676034279987523,-10.200171,-17.323240,-2.689153,0.006686,0.016921,0.394677,0.918640,[],['KB']
4,1676034285358658,2023-02-10-08-04-19-twilight,1676034285387661,1676034285339092,-6.752502,-13.812644,-2.623646,0.003145,0.013536,0.374985,0.926927,[],['KE']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,1678905610167111,2023-03-15-13-25-48-night,1678905610166555,1678905610148946,-208.692754,19.746243,-2.661155,0.010751,0.021335,0.389329,0.920789,['АРКТИКА'],[]
1355,1678905615814276,2023-03-15-13-25-48-night,1678905615783690,1678905615831108,-204.946252,23.201232,-2.763870,-0.001526,0.012262,0.270024,0.962774,['АРКТИКА'],[]
1356,1678905621059160,2023-03-15-13-25-48-night,1678905621067311,1678905621028555,-200.439570,25.320586,-2.827866,0.000658,0.011407,0.146494,0.989146,['АРКТИКА'],[]
1357,1678905626909006,2023-03-15-13-25-48-night,1678905626884315,1678905626895739,-195.479406,26.327213,-2.666329,0.007004,-0.019667,-0.149021,0.988614,['АВТОМАТ'],[]


In [ ]:
test_df = pd.read_csv('/kaggle/input/itlp-campus-dataset/public/test_with_rec_text.csv').drop(columns='Unnamed: 0')
test_df

,timestamp,track,front_cam_ts,back_cam_ts,tx,ty,tz,qx,qy,qz,qw,front_cam_text,back_cam_text
0,1676034260851770,2023-02-10-08-04-19-twilight,1676034260821540,1676034260865870,-23.839826,-17.655232,-1.474530,0.013051,0.028187,0.003870,0.999510,[],[]
1,1676034268718093,2023-02-10-08-04-19-twilight,1676034268687989,1676034268708533,-19.060815,-19.003152,-2.513494,0.017344,0.019243,-0.121944,0.992199,[],[]
2,1676034274365801,2023-02-10-08-04-19-twilight,1676034274404483,1676034274382407,-14.289875,-20.252643,-2.563523,0.019104,0.025107,0.103990,0.994078,[],['KE']
3,1676034280013503,2023-02-10-08-04-19-twilight,1676034280004376,1676034279987523,-10.200171,-17.323240,-2.689153,0.006686,0.016921,0.394677,0.918640,[],['KB']
4,1676034285358658,2023-02-10-08-04-19-twilight,1676034285387661,1676034285339092,-6.752502,-13.812644,-2.623646,0.003145,0.013536,0.374985,0.926927,[],['KE']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,1678905610167111,2023-03-15-13-25-48-night,1678905610166555,1678905610148946,-208.692754,19.746243,-2.661155,0.010751,0.021335,0.389329,0.920789,['АРКТИКА'],[]
1851,1678905615814276,2023-03-15-13-25-48-night,1678905615783690,1678905615831108,-204.946252,23.201232,-2.763870,-0.001526,0.012262,0.270024,0.962774,['АРКТИКА'],[]
1852,1678905621059160,2023-03-15-13-25-48-night,1678905621067311,1678905621028555,-200.439570,25.320586,-2.827866,0.000658,0.011407,0.146494,0.989146,['АРКТИКА'],[]
1853,1678905626909006,2023-03-15-13-25-48-night,1678905626884315,1678905626895739,-195.479406,26.327213,-2.666329,0.007004,-0.019667,-0.149021,0.988614,['АВТОМАТ'],[]


In [ ]:
val_df = pd.read_csv('/kaggle/input/itlp-campus-dataset/public/val.csv').drop(columns='Unnamed: 0')
val_df

,timestamp,track,front_cam_ts,back_cam_ts,tx,ty,tz,qx,qy,qz,qw,front_cam_text,back_cam_text
0,1676035566075019,2023-02-10-08-04-19-twilight,1676035566108253,1676035566077315,80.724531,-71.308620,-0.873446,-0.000971,0.007920,0.327289,0.944891,['ФИЗТЕХ.БИО'],"['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС', '1945', '1957']"
1,1676035571722667,2023-02-10-08-04-19-twilight,1676035571691405,1676035571749351,84.696672,-67.969684,-0.886570,-0.000097,0.017570,0.324952,0.945567,['ФИЗТЕХ.Б'],"['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС', '1952']"
2,1676035577269474,2023-02-10-08-04-19-twilight,1676035577291312,1676035577284949,88.486198,-64.722221,-0.878774,-0.005348,0.022473,0.352585,0.935495,['РАДИОТЕХНИЧЕСКИЙ ФАКУЛЬТЕТ'],"['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС']"
3,1676035582715506,2023-02-10-08-04-19-twilight,1676035582724463,1676035582747679,92.378366,-61.577601,-0.906462,-0.004888,0.020862,0.231267,0.972654,"['ФИЗТЕХ', 'РАДИОТЕХНИЧЕСКИЙ', 'КОРПУС']","['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС']"
4,1676035588262317,2023-02-10-08-04-19-twilight,1676035588290997,1676035588289057,96.279563,-58.386627,-1.030017,-0.007974,0.017085,0.382182,0.923895,"['РАДИОТЕХНИЧЕСКИЙ', 'КОРПУС', 'P']","['МФТИ', 'ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,1678904283512023,2023-03-15-13-25-48-night,1678904283555720,1678904283534244,-155.078201,-194.151665,-1.393639,-0.024897,-0.018326,0.741418,0.670331,['ФИЗТ'],[]
492,1678904290067940,2023-03-15-13-25-48-night,1678904290056078,1678904290072465,-154.392377,-189.269135,-1.392422,-0.013549,0.009834,0.618619,0.785512,['ФИЗТЕХ. КВАНТ'],[]
493,1678904296724738,2023-03-15-13-25-48-night,1678904296739853,1678904296744134,-153.911107,-184.344878,-1.207735,-0.001541,0.003475,0.649004,0.760775,['Физтех'],[]
494,1678904302373180,2023-03-15-13-25-48-night,1678904302356838,1678904302348016,-153.259422,-179.365098,-1.463031,0.003824,0.010529,0.686563,0.726984,[],[]


In [ ]:
!pip install git+https://github.com/alexmelekhin/open_place_recognition.git@feat/itlp_outdoor

In [ ]:
!pip install gdown

In [ ]:
from torch.utils.data import DataLoader

from opr.datasets.itlp_outdoor import ITLPCampusOutdoor
from opr.samplers import BatchSampler

In [ ]:
# семантические маски
dd = {
  "stuff_classes": [
    "Bird",
    "Ground Animal",
    "Curb",
    "Fence",
    "Guard Rail",
    "Barrier",
    "Wall",
    "Bike Lane",
    "Crosswalk - Plain",
    "Curb Cut",
    "Parking",
    "Pedestrian Area",
    "Rail Track",
    "Road",
    "Service Lane",
    "Sidewalk",
    "Bridge",
    "Building",
    "Tunnel",
    "Person",
    "Bicyclist",
    "Motorcyclist",
    "Other Rider",
    "Lane Marking - Crosswalk",
    "Lane Marking - General",
    "Mountain",
    "Sand",
    "Sky",
    "Snow",
    "Terrain",
    "Vegetation",
    "Water",
    "Banner",
    "Bench",
    "Bike Rack",
    "Billboard",
    "Catch Basin",
    "CCTV Camera",
    "Fire Hydrant",
    "Junction Box",
    "Mailbox",
    "Manhole",
    "Phone Booth",
    "Pothole",
    "Street Light",
    "Pole",
    "Traffic Sign Frame",
    "Utility Pole",
    "Traffic Light",
    "Traffic Sign (Back)",
    "Traffic Sign (Front)",
    "Trash Can",
    "Bicycle",
    "Boat",
    "Bus",
    "Car",
    "Caravan",
    "Motorcycle",
    "On Rails",
    "Other Vehicle",
    "Trailer",
    "Truck",
    "Wheeled Slow",
    "Car Mount",
    "Ego Vehicle"
  ],
  "stuff_colors": [
    [165, 42, 42],
    [0, 192, 0],
    [196, 196, 196],
    [190, 153, 153],
    [180, 165, 180],
    [90, 120, 150],
    [102, 102, 156],
    [128, 64, 255],
    [140, 140, 200],
    [170, 170, 170],
    [250, 170, 160],
    [96, 96, 96],
    [230, 150, 140],
    [128, 64, 128],
    [110, 110, 110],
    [244, 35, 232],
    [150, 100, 100],
    [70, 70, 70],
    [150, 120, 90],
    [220, 20, 60],
    [255, 0, 0],
    [255, 0, 100],
    [255, 0, 200],
    [200, 128, 128],
    [255, 255, 255],
    [64, 170, 64],
    [230, 160, 50],
    [70, 130, 180],
    [190, 255, 255],
    [152, 251, 152],
    [107, 142, 35],
    [0, 170, 30],
    [255, 255, 128],
    [250, 0, 30],
    [100, 140, 180],
    [220, 220, 220],
    [220, 128, 128],
    [222, 40, 40],
    [100, 170, 30],
    [40, 40, 40],
    [33, 33, 33],
    [100, 128, 160],
    [142, 0, 0],
    [70, 100, 150],
    [210, 170, 100],
    [153, 153, 153],
    [128, 128, 128],
    [0, 0, 80],
    [250, 170, 30],
    [192, 192, 192],
    [220, 220, 0],
    [140, 140, 20],
    [119, 11, 32],
    [150, 0, 255],
    [0, 60, 100],
    [0, 0, 142],
    [0, 0, 90],
    [0, 0, 230],
    [0, 80, 100],
    [128, 64, 64],
    [0, 0, 110],
    [0, 0, 70],
    [0, 0, 192],
    [32, 32, 32],
    [120, 10, 10]
  ]
}

In [ ]:
stuff_classes = dd['stuff_classes']
stuff_colors = dd['stuff_colors']

In [ ]:
# кастомная функция для того, чтобы убрать динамические объекты
import numpy as np
import cv2
from typing import *

def remove_dynamic_objects(mask_image: np.ndarray, stuff_classes: List[str], stuff_colors: List[Tuple[int, int, int]]) -> np.ndarray:
    dynamic_classes = ["Person", "Bicyclist", "Motorcyclist", "Other Rider",
        "Car", "Bus", "Motorcycle", "On Rails", "Other Vehicle",
        "Trailer", "Truck", "Wheeled Slow", "Car Mount", "Ego Vehicle"]

    dynamic_mask = np.zeros(mask_image.shape[:2], dtype=np.uint8)
    for idx, class_name in enumerate(stuff_classes):
        if class_name in dynamic_classes:
            color = stuff_colors[idx]
            class_mask = cv2.inRange(mask_image, np.array(color), np.array(color))
            dynamic_mask = cv2.bitwise_or(dynamic_mask, class_mask)
    static_mask = cv2.bitwise_not(dynamic_mask)
    result_mask = cv2.bitwise_and(mask_image, mask_image, mask=static_mask)
    return result_mask
    return result_mask

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

In [ ]:
class E5Embedder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
        self.model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

    def average_pool(self, last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def forward(self, text):
        tokenized = self.tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
        outputs = self.model(**tokenized)
        embeddings = self.average_pool(outputs.last_hidden_state, tokenized['attention_mask'])
        return embeddings

In [ ]:
"""Custom ITLP-Campus dataset implementations."""
import math
from pathlib import Path
from typing import Dict, List, Literal, Optional, Tuple, Union

import cv2
import gdown
import numpy as np
import pandas as pd
import torch
from loguru import logger
from omegaconf import OmegaConf
from pandas import DataFrame
from torch import Tensor
from torch.utils.data import Dataset

from opr.datasets.augmentations import (
    DefaultCloudSetTransform,
    DefaultCloudTransform,
    DefaultImageTransform,
    DefaultSemanticTransform,
)


class ITLPCampusOutdoor(Dataset):
    dataset_root: Path
    dataset_df: DataFrame
    front_cam_text_labels_df: Optional[DataFrame]
    back_cam_text_labels_df: Optional[DataFrame]
    sensors: Tuple[str, ...]
    images_subdir: str = ""
    clouds_subdir: str = "lidar"
    semantic_subdir: str = "masks"
    image_transform: DefaultImageTransform
    pointcloud_transform: DefaultCloudTransform
    cloud_set_transform: DefaultCloudSetTransform
    _pointcloud_quantization_size: Optional[float]
    load_semantics: bool
    load_text_labels: bool
    cam_config: dict
    sensors_cfg: OmegaConf
    top_k_soc: int
    soc_coords_type: Literal["cylindrical_3d", "cylindrical_2d", "euclidean", "spherical"] = "cylindrical_3d"
    max_distance_soc: float
    anno: OmegaConf
    subset: Literal["train", "val", "test"]
    vis_dir: str = "./vis/"
    train_split: list = None
    test_split: list = None

    def __init__(
        self,
        dataset_root: Union[str, Path],
        subset: Literal["train", "val", "test"],
        sensors: Union[str, Tuple[str, ...]] = ("front_cam", "back_cam"),
        mink_quantization_size: Optional[float] = 0.5,
        max_point_distance: Optional[float] = None,
        load_semantics: bool = False,
        load_text_labels: bool = False,
        positive_threshold: float = 10.0,
        negative_threshold: float = 50.0,
        image_transform = DefaultImageTransform(resize=(320, 192), train=False),
        semantic_transform = DefaultSemanticTransform(resize=(320, 192), train=False),
    ) -> None:

        super().__init__()
        self.dataset_root = Path(dataset_root)
        if not self.dataset_root.exists():
            raise FileNotFoundError(f"Given dataset_root={self.dataset_root} doesn't exist")

        self.subset = subset
        if self.subset not in ("train", "val", "test", "test_with_rec_text"):
            raise ValueError(f"Unknown subset: {subset!r}")

        subset_csv = self.dataset_root / f"{self.subset}.csv"
        self.dataset_df = pd.read_csv(subset_csv)

        if self.subset == "test":
            self.dataset_df["in_query"] = True

        if isinstance(sensors, str):
            sensors = tuple(sensors)
        self.sensors = sensors

        self.load_text_labels = load_text_labels

        self._pointcloud_quantization_size = mink_quantization_size
        self._max_point_distance = max_point_distance
        self.load_semantics = load_semantics

        self.embedder = E5Embedder()

        if positive_threshold < 0.0:
            raise ValueError(f"positive_threshold must be non-negative, but {positive_threshold!r} given.")
        if negative_threshold < 0.0:
            raise ValueError(f"negative_threshold must be non-negative, but {negative_threshold!r} given.")

        self._positives_index, self._nonnegative_index = self._build_indexes(
            positive_threshold, negative_threshold
        )
        self._positives_mask, self._negatives_mask = self._build_masks(positive_threshold, negative_threshold)

        self.image_transform = image_transform
        self.semantic_transform = semantic_transform
        self.pointcloud_transform = DefaultCloudTransform(train=False)
        self.pointcloud_set_transform = DefaultCloudSetTransform(train=False)

    def _load_image(self, cam: str, idx: int, track: str, floor: str, transform: bool = True) -> Tensor:
        image_ts = int(self.dataset_df[f"{cam}_ts"].iloc[idx])
        im_filepath = self.dataset_root / track / floor / self.images_subdir / cam / f"{image_ts}.png"
        im = cv2.imread(str(im_filepath))
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        if transform:
            im = self.image_transform(im)
        return im

    def _load_semantic_mask(
        self, cam: str, idx: int, track: str, floor: str, transform: bool = True
    ) -> np.ndarray:
        mask_ts = int(self.dataset_df[f"{cam}_ts"].iloc[idx])
        im_filepath = self.dataset_root / track / floor / self.semantic_subdir / cam / f"{mask_ts}.png"
        im = cv2.imread(str(im_filepath))
        if transform:
            im = self.semantic_transform(im)
        return im

    def __getitem__(self, idx: int) -> Dict[str, Union[int, Tensor]]:  # noqa: D105
        data: Dict[str, Union[int, Tensor]] = {"idx": torch.tensor(idx)}
        data["pose"] = torch.tensor(
            self.dataset_df.iloc[idx][["tx", "ty", "tz", "qx", "qy", "qz", "qw"]].to_numpy(dtype=np.float32)
        )
        floor = self._get_floor_subdir(idx)
        track = self._get_track_subdir(idx)

        if "front_cam" in self.sensors:
            im = self._load_image("front_cam", idx, track, floor)
            data["image_front_cam"] = im
            if self.load_semantics:
                im = self._load_semantic_mask("front_cam", idx, track, floor)
                data["mask_front_cam"] = im
            if self.load_text_labels:
                data["text_front_cam"] = self.embedder.forward(self.dataset_df["front_cam_text"].iloc[idx])

        if "back_cam" in self.sensors:
            im = self._load_image("back_cam", idx, track, floor)
            data["image_back_cam"] = im
            if self.load_semantics:
                im = self._load_semantic_mask("back_cam", idx, track, floor)
            if self.load_text_labels:
                data["text_back_cam"] = self.embedder.forward(self.dataset_df["back_cam_text"].iloc[idx])

        if "lidar" in self.sensors:
            pc = self._load_pc(idx, track, floor)

            pc = torch.tensor(pc, dtype=torch.float32)
            data["pointcloud_lidar_coords"] = pc
            data["pointcloud_lidar_feats"] = torch.ones_like(pc[:, :1])

        return data

    def __len__(self) -> int:  # noqa: D105
        return len(self.dataset_df)

    def _get_floor_subdir(self, idx: int) -> str:
        if "floor" in self.dataset_df.columns:
            return f"floor_{self.dataset_df['floor'].iloc[idx]}"
        else:
            return ""

    def _get_track_subdir(self, idx: int) -> str:
        if "track" in self.dataset_df.columns:
            return self.dataset_df["track"].iloc[idx]
        else:
            return ""

    def _load_pc(self, idx: int, track: str, floor: str, tensor: bool = True) -> Tensor:
        lidar_ts = int(self.dataset_df["lidar_ts"].iloc[idx])
        filepath = self.dataset_root / track / floor / self.clouds_subdir / f"{lidar_ts}.bin"
        pc = np.fromfile(filepath, dtype=np.float32).reshape((-1, 4))[:, :-1]
        in_range_idx = np.all(
            np.logical_and(-100 <= pc, pc <= 100),
            axis=1,
        )
        pc = pc[in_range_idx]
        if self._max_point_distance is not None:
            pc = pc[np.linalg.norm(pc, axis=1) < self._max_point_distance]
        if tensor:
            pc = torch.tensor(pc, dtype=torch.float32)
        return pc

    def _collate_data_dict(self, data_list: List[Dict[str, Tensor]]) -> Dict[str, Tensor]:
        result: Dict[str, Tensor] = {}
        result["idxs"] = torch.stack([e["idx"] for e in data_list], dim=0)
        for data_key in data_list[0].keys():
            if data_key == "idx":
                continue
            elif data_key == "text_front_cam":
                result["text_front_cam"] = torch.stack([e["text_front_cam"] for e in data_list], dim=0)
            elif data_key == "text_back_cam":
                result["text_back_cam"] = torch.stack([e["text_back_cam"] for e in data_list], dim=0)
            elif data_key == "pose":
                result["poses"] = torch.stack([e["pose"] for e in data_list], dim=0)
            elif data_key.startswith("image_"):
                result[f"images_{data_key[6:]}"] = torch.stack([e[data_key] for e in data_list])
            elif data_key.startswith("mask_"):
                result[f"masks_{data_key[5:]}"] = torch.stack([e[data_key] for e in data_list])
            elif data_key == "pointcloud_lidar_coords":
                if not minkowski_available:
                    raise RuntimeError("MinkowskiEngine is not installed. Cannot process point clouds.")
                coords_list = [e["pointcloud_lidar_coords"] for e in data_list]
                feats_list = [e["pointcloud_lidar_feats"] for e in data_list]
                n_points = [int(e.shape[0]) for e in coords_list]
                coords_tensor = torch.cat(coords_list, dim=0).unsqueeze(0)  # (1,batch_size*n_points,3)
                if self.pointcloud_set_transform is not None:
                    coords_tensor = self.pointcloud_set_transform(coords_tensor)
                coords_list = torch.split(
                    coords_tensor.squeeze(0),
                    split_size_or_sections=n_points,
                    dim=0,
                )
                quantized_coords_list = []
                quantized_feats_list = []
                for coords, feats in zip(coords_list, feats_list):
                    (
                        quantized_coords,
                        quantized_feats,
                    ) = ME.utils.sparse_quantize(
                        coordinates=coords,
                        features=feats,
                        quantization_size=self._pointcloud_quantization_size,
                    )
                    quantized_coords_list.append(quantized_coords)
                    quantized_feats_list.append(quantized_feats)

                result["pointclouds_lidar_coords"] = ME.utils.batched_coordinates(quantized_coords_list)
                result["pointclouds_lidar_feats"] = torch.cat(quantized_feats_list)
            elif data_key == "pointcloud_lidar_feats":
                continue
            else:
                raise ValueError(f"Unknown data key: {data_key!r}")
        return result

    def collate_fn(self, data_list: List[Dict[str, Tensor]]) -> Dict[str, Tensor]:
        return self._collate_data_dict(data_list)


    def _build_masks(self, positive_threshold: float, negative_threshold: float) -> Tuple[Tensor, Tensor]:
        xyz = torch.tensor(
            self.dataset_df[["tx", "ty", "tz"]].to_numpy(dtype=np.float32),
            dtype=torch.float32,
        )
        distances = torch.cdist(xyz, xyz)

        positives_mask = (distances > 0) & (distances < positive_threshold)
        negatives_mask = distances > negative_threshold

        return positives_mask, negatives_mask

    def _build_indexes(
        self, positive_threshold: float, negative_threshold: float
    ) -> Tuple[List[Tensor], List[Tensor]]:
        xyz = torch.tensor(
            self.dataset_df[["tx", "ty", "tz"]].to_numpy(dtype=np.float32),
            dtype=torch.float32,
        )
        distances = torch.cdist(xyz, xyz)

        positives_mask = (distances > 0) & (distances < positive_threshold)
        nonnegatives_mask = distances < negative_threshold

        positive_indices = [torch.nonzero(row).squeeze(dim=-1) for row in positives_mask]
        nonnegative_indices = [torch.nonzero(row).squeeze(dim=-1) for row in nonnegatives_mask]

        return positive_indices, nonnegative_indices

    @property
    def positives_index(self) -> List[Tensor]:
        return self._positives_index

    @property
    def nonnegative_index(self) -> List[Tensor]:
        return self._nonnegative_index

    @property
    def positives_mask(self) -> Tensor:
        return self._positives_mask

    @property
    def negatives_mask(self) -> Tensor:
        return self._negatives_mask

In [ ]:
train_dataset = ITLPCampusOutdoor(
    DATA_ROOT,
    subset='train',
    sensors=('front_cam', 'back_cam'),
    load_semantics=False,
    load_text_labels=True,
    mink_quantization_size=0.06,
)

val_dataset = ITLPCampusOutdoor(
    DATA_ROOT,
    subset='val',
    sensors=('front_cam', 'back_cam'),
    load_semantics=False,
    load_text_labels=True,
    mink_quantization_size=0.06,
)

test_dataset = ITLPCampusOutdoor(
    DATA_ROOT,
    subset='test_with_rec_text',
    sensors=('front_cam', 'back_cam'),
    load_semantics=False,
    load_text_labels=True,
    mink_quantization_size=0.06,
)

In [ ]:
test_dataset[600]

{'idx': tensor(600),
 'pose': tensor([-2.2312e+02,  5.5457e+00, -2.3279e+00,  7.6497e-03,  3.0006e-02,
          3.3897e-01,  9.4029e-01]),
 'image_front_cam': tensor([[[ 1.0844,  1.0844,  1.0673,  ...,  1.1529,  1.1358,  1.1358],
          [ 1.0844,  1.0844,  1.0673,  ...,  1.1529,  1.1358,  1.1358],
          [ 1.0844,  1.0844,  1.0844,  ...,  1.1529,  1.1358,  1.1358],
          ...,
          [-1.3473, -1.3644, -1.3130,  ..., -1.0562, -1.0904, -0.9705],
          [-1.3644, -1.3815, -1.3815,  ..., -1.0733, -1.0219, -1.0562],
          [-1.3644, -1.3815, -1.3644,  ..., -1.0048, -1.0048, -0.9705]],
 
         [[ 1.8158,  1.8158,  1.7983,  ...,  1.8508,  1.8333,  1.8333],
          [ 1.8158,  1.8158,  1.7983,  ...,  1.8508,  1.8333,  1.8333],
          [ 1.8158,  1.8158,  1.8158,  ...,  1.8508,  1.8333,  1.8333],
          ...,
          [-1.1604, -1.1779, -1.1253,  ..., -0.8452, -0.8978, -0.7752],
          [-1.1779, -1.1954, -1.1954,  ..., -0.8803, -0.8277, -0.8627],
          [-1.17

In [ ]:
train_sampler = BatchSampler(
    train_dataset,
    batch_size=16,
    batch_size_limit=256,
    batch_expansion_rate=1.4,
    drop_last=True,
)

val_sampler = BatchSampler(
    val_dataset,
    batch_size=256,
    drop_last=True,
)

In [ ]:
train_dl = DataLoader(
    dataset=train_dataset,
    batch_sampler=train_sampler,
    collate_fn=train_dataset.collate_fn,
    pin_memory=True,
)

val_dl = DataLoader(
    dataset=val_dataset,
    batch_sampler=val_sampler,
    collate_fn=val_dataset.collate_fn,
    pin_memory=True,
)

test_dl = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    collate_fn=test_dataset.collate_fn,
    pin_memory=True,
    drop_last=False,
)

In [ ]:
import torch
from opr.models.place_recognition.base import ImageModel, LateFusionModel
from opr.modules.feature_extractors import ConvNeXtTinyFeatureExtractor
from opr.modules import Concat, GeM

In [ ]:
from torch import Tensor, nn

class Attention(nn.Module):
    def __init__(self, image_dim, text_dim, hidden_dim):
        super(Attention, self).__init__()
        self.image_linear = nn.Linear(image_dim, hidden_dim)
        self.text_linear = nn.Linear(text_dim, hidden_dim)
        self.attention_linear = nn.Linear(hidden_dim, 1)

    def forward(self, images: Tensor, texts: Tensor) -> Tensor:
        proj_images = self.image_linear(images)
        proj_texts = self.text_linear(texts)

        combined = torch.tanh(proj_images.unsqueeze(1) + proj_texts.unsqueeze(2))
        attention_scores = self.attention_linear(combined).squeeze(2)

        attention_weights = F.softmax(attention_scores, dim=1).unsqueeze(2)

        weighted_image = torch.sum(attention_weights * images.unsqueeze(1), dim=1)
        weighted_text = torch.sum(attention_weights * texts.unsqueeze(1), dim=1)

        return weighted_image, weighted_text

In [ ]:
class LateFusionModel(nn.Module):
    def __init__(
        self,
        image_module,
        text_module,
        fusion_module=None,
        semantic_module=None
    ) -> None:
        super().__init__()

        self.image_module = image_module
        self.text_module = text_module
        self.semantic_module = semantic_module

        if fusion_module:
            self.fusion_module = fusion_module
        else:
            self.fusion_module = Attention(image_module.output_dim, text_module.output_dim, hidden_dim=128)

    def forward(self, batch: Dict[str, Tensor]) -> Dict[str, Tensor]:
        out_dict: Dict[str, Tensor] = {}

        if self.image_module is not None:
            out_dict["image"] = self.image_module(batch)["final_descriptor"]

        if self.text_module is not None:
            out_dict["text"] = self.text_module(batch)["final_descriptor"]

        if self.semantic_module is not None:
            out_dict["semantic"] = self.semantic_module(batch)["final_descriptor"]

        if 'image' in out_dict and 'text' in out_dict:
            fused_image, fused_text = self.fusion_module({'image': out_dict["image"], 'image': out_dict["text"]})
            out_dict["fused_image"] = fused_image
            out_dict["fused_text"] = fused_text

        return out_dict

In [ ]:
class Dummy:
    def __init__(self):
        pass

    def __call__(self, emb):
        return {"final_descriptor": emb}

    def forward(self, emb):
        return {"final_descriptor": emb}

In [ ]:
pooling = GeM()
concat = Concat()

feature_extractor = ConvNeXtTinyFeatureExtractor(
    in_channels=3,
    pretrained=True,
)

text_feature_extractor = Dummy()

image_module = ImageModel(
    backbone=feature_extractor,
    head=pooling,
    fusion=concat,
)

model = LateFusionModel(
    image_module=image_module,
    text_module=text_feature_extractor,
    fusion_module=concat
)

In [ ]:
IMAGE_LR = 0.003
HEAD_LR = None
FUSION_LR = None
WEIGHT_DECAY = 0.003
WARMUP_STEPS = 90
T_MAX = 1800

In [ ]:
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

In [ ]:
params_list = []
if model.image_module is not None and IMAGE_LR is not None:
    params_list.append({"params": model.image_module.parameters(), "lr": IMAGE_LR})
if model.fusion_module is not None and FUSION_LR is not None:
    params_list.append({"params": model.fusion_module.parameters(), "lr": FUSION_LR})

In [ ]:
from opr.trainers.place_recognition import MultimodalPlaceRecognitionTrainer
from opr.losses import BatchHardTripletMarginLoss

In [ ]:
!mkdir checkpoints

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mkdir: cannot create directory 'checkpoints': File exists


In [ ]:
loss_fn = BatchHardTripletMarginLoss(margin=0.5)
optimizer = AdamW(params_list, weight_decay=WEIGHT_DECAY)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=T_MAX)

In [ ]:
kwargs = {
    'checkpoints_dir': '/kaggle/working/checkpoints',
    'model': model,
    'loss_fn': loss_fn,
    'optimizer': optimizer,
    'scheduler': scheduler,
    'batch_expansion_threshold': 0.7,
    'wandb_log': False,
    'device': 'cuda',
}

In [ ]:
trainer = MultimodalPlaceRecognitionTrainer(
    modalities_weights={
        "image": 0.9,
        "cloud": 0,
        "semantic": 0.05,
        "text": 0.05,
        "final_descriptor": 0
    },
    **kwargs
)

In [ ]:
# trainer = UnimodalPlaceRecognitionTrainer(
#     checkpoints_dir='/kaggle/working/checkpoints',
#     model=model,
#     loss_fn=loss_fn,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     batch_expansion_threshold=0.7,
#     wandb_log=False,
#     device='cuda',
# )

In [ ]:
trainer.train(
    epochs=6,
    train_dataloader=train_dl,
    val_dataloader=val_dl,
    test_dataloader=test_dl,
)

2024-07-03 15:43:13.616 | INFO     | opr.trainers.place_recognition.unimodal:train:113 - =====> Epoch:   1/6:
2024-07-03 15:43:13.617 | INFO     | opr.trainers.place_recognition.multimodal:_loop_epoch:34 - => Train stage:


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 34.12 MiB is free. Process 2372 has 15.86 GiB memory in use. Of the allocated memory 15.17 GiB is allocated by PyTorch, and 403.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.test(test_dl, distance_threshold=5)

2024-07-03 15:51:49.097 | INFO     | opr.trainers.place_recognition.unimodal:test:172 - => Test stage:


OutOfMemoryError: CUDA out of memory. Tried to allocate 180.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 34.12 MiB is free. Process 2372 has 15.86 GiB memory in use. Of the allocated memory 15.17 GiB is allocated by PyTorch, and 403.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
!pip install faiss-cpu

In [ ]:
import itertools
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import faiss

def extract_embeddings(model, descriptor_key, dataloader, device):
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        test_embeddings_list = []
        for data in tqdm(dataloader, desc="Calculating test set descriptors"):
            batch = data
            batch = {e: batch[e].to(device) for e in batch}
            batch_embeddings = model(batch)
            test_embeddings_list.append(batch_embeddings[descriptor_key].cpu().numpy())
        test_embeddings = np.vstack(test_embeddings_list)
    return test_embeddings

def test_submission(
    test_embeddings: np.ndarray, dataset_df: pd.DataFrame, filename: str = "submission.txt"
) -> None:
    """Function to create submission txt file.

    Args:
        test_embeddings (np.ndarray): Array of embeddings.
        dataset_df (pd.Dataframe): Test dataset dataframe ('test.csv').
        filename (str): Name of the output txt file. Defaults to "submission.txt".
    """
    tracks = []

    for _, group in dataset_df.groupby("track"):
        tracks.append(group.index.to_numpy())
    n = 1
    ij_permutations = sorted(list(itertools.permutations(range(len(tracks)), 2)))
    # ij_permutations = [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]

    submission_lines = []

    for i, j in tqdm(ij_permutations, desc="Calculating metrics"):
        query_indices = tracks[i]
        database_indices = tracks[j]
        query_embs = test_embeddings[query_indices]
        database_embs = test_embeddings[database_indices]

        dimension = database_embs.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(database_embs)

        _, indices = index.search(query_embs, n)

        submission_lines.extend(list(database_indices[indices.squeeze()]))

    with open(filename, "w") as f:
        for l in submission_lines:
            f.write(str(l)+"\n")

In [ ]:
embeddings = extract_embeddings(model, descriptor_key="final_descriptor", dataloader=test_dl, device='cuda')
test_submission(embeddings, dataset_df=test_dl.dataset.dataset_df, filename="tuned_baseline_submission.txt")